<img src="https://app.powerbi.com/13.0.10537.111/images/PowerBI96x96.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="10%"> = ❤

# Export Power BI Workspace Assets with PowerShell

This PowerShell Notebook will help you collect information about the following types of Power BI assets, from the Power BI service:

+ Workspaces
+ Datasets
+ Reports
+ Power BI dataflows
+ Data Sources of those Power BI dataflows

NOTE: This Notebook exports all the this information to different Worksheets within the same Excel file.  In order to do this, we use the community-driven `ImportExcel` module from the PowerShell Gallery.  Also of note, the name of the Excel file we are exporting to is named `c:\temp\PBIServiceAssets.xlsx` but you may want to change that.

Before you can do anything with any of the Power BI cmdlets, you must first login to the Power BI Service.

First, collect info about all the **Workspaces**, across your entire tenant.

NOTE: You must be a Power BI Admin in order to successfully execute the code below.  If you are not a Power BI Admin, remove the `-Scope Organization` portion of the code below, as well as all subsequent commands.

In [ ]:
Get-PowerBIWorkspace -Scope Organization -All |
Export-Excel -Path C:\temp\DataGovernance-for-PowerBI\PBIServiceAssets.xlsx -WorksheetName Workspace

Collect all of the **Datasets** across the tenant and append the WorkspaceID they belong to.

In [ ]:
foreach ($Workspace in Get-PowerBIWorkspace -Scope Organization -All) 
{
    Get-PowerBIDataset -WorkspaceId $Workspace.Id -Scope Organization |
    Export-Excel -Path c:\temp\DataGovernance-for-PowerBI\PBIServiceAssets.xlsx -WorksheetName DatasetsIn_Workspaces -Append
}

Collect all of the **Reports** across the tenant and append the WorkspaceID they belong to.

In [ ]:
foreach ($Workspace in Get-PowerBIWorkspace -Scope Organization -All ) 
{
    $Dataset = Get-PowerBIReport -WorkspaceId ($Workspace.Id) -Scope Organization
    $Dataset | Add-Member -NotePropertyName WorkspaceId -NotePropertyValue $Workspace.Id;
    $Dataset | 
    Export-Excel -Path c:\temp\DataGovernance-for-PowerBI\PBIServiceAssets.xlsx -WorksheetName ReportsIn_Workspaces -Append
}

Collect all of the **dataflows** across the tenant and append the WorkspaceID they belong to.

In [ ]:
foreach ($Workspace in Get-PowerBIWorkspace -Scope Organization -All | WHERE { $_.Type -eq 'Workspace'}) 
{
    $Dataflow = Get-PowerBIDataflow -WorkspaceId $Workspace.Id -Scope Organization 
    $Dataflow | Add-Member -NotePropertyName WorkspaceId -NotePropertyValue $Workspace.Id;
    $Dataflow | 
    Export-Excel -Path c:\temp\DataGovernance-for-PowerBI\PBIServiceAssets.xlsx -WorksheetName dataflows -Append
}

Collect all the **Datasources** of the **dataflows** across the tenant and append the WorkspaceID they belong to.
NOTE: There used to be a bug in the API that causes this step to time out on dataflows which have never been 'refreshed'.  I think it has been fixed, but please let me know if you're still running into it?

In [ ]:
foreach ($Workspace in Get-PowerBIWorkspace -Scope Organization | WHERE { $_.Type -eq 'Workspace'} ) 
{
    $DataflowDatasource = Get-PowerBIDataflow -WorkspaceId $Workspace.Id -Scope Organization | 
    Get-PowerBIDataflowDatasource -WorkspaceId $Workspace.Id -Scope Organization
    $DataflowDatasource | Add-Member -NotePropertyName WorkspaceId -NotePropertyValue $Workspace.Id;
    $DataflowDatasource | 
    Export-Excel -Path c:\temp\DataGovernance-for-PowerBI\PBIServiceAssets.xlsx -WorksheetName dataflowDatasources -Append
}

## You're done.  Go import your data 👍

If you're interested in automating this.  You can automate PowerShell scripts with a SQL Server Agent Job.